In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/traffic-signs-detection-system/split_dataset/split_dataset/labels/train.cache
/kaggle/input/traffic-signs-detection-system/split_dataset/split_dataset/labels/val/567623821789320.txt
/kaggle/input/traffic-signs-detection-system/split_dataset/split_dataset/labels/train/291886865908621.txt
/kaggle/input/traffic-signs-detection-system/split_dataset/split_dataset/images/val/729859605883973.jpg
/kaggle/input/traffic-signs-detection-system/split_dataset/split_dataset/images/train/142269081251480.jpg
/kaggle/input/traffic-signs-detection-system/total_dataset/total_dataset/labels/val/567623821789320.txt
/kaggle/input/traffic-signs-detection-system/total_dataset/total_dataset/labels/train/291886865908621.txt
/kaggle/input/traffic-signs-detection-system/total_dataset/total_dataset/images/val/729859605883973.jpg
/kaggle/input/traffic-signs-detection-system/total_dataset/total_dataset/images/train/142269081251480.jpg


In [2]:
!pip install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
!pip install -U albumentations
!pip install -U git+https://github.com/ultralytics/yolov5.git

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# import zipfile
# import os

# with zipfile.ZipFile('/kaggle/input/traffic-signs-detection-system', 'r') as zip_ref:
#     zip_ref.extractall('/kaggle/working/')


In [4]:
# Clone the YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 16977, done.
remote: Counting objects: 100% (172/172), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 16977 (delta 90), reused 111 (delta 57), pack-reused 16805 (from 1)
Receiving objects: 100% (16977/16977), 15.72 MiB | 30.42 MiB/s, done.
Resolving deltas: 100% (11623/11623), done.


In [5]:
# Change directory to yolov5
%cd yolov5

# Install required packages
!pip install -r requirements.txt


/kaggle/working/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 882.8/882.8 kB 21.5 MB/s eta 0:00:00


In [6]:
data_yaml = """
train: /kaggle/input/traffic-signs-detection-system/total_dataset/total_dataset/images/train
val: /kaggle/input/traffic-signs-detection-system/total_dataset/total_dataset/images/train

nc: 8
names: ['regulatory--no-parking--g2', 'warning--pedestrians-crossing--g5', 'regulatory--no-entry--g1', 'regulatory--no-right-turn--g1', 'regulatory--no-left-turn--g2', 'regulatory--no-horn--g1', 'regulatory--no-stopping--g1', 'information--pedestrians-crossing--g1']
"""

with open('/kaggle/working/data.yaml', 'w') as f:
    f.write(data_yaml)


In [7]:
import os

train_dir = '/kaggle/input/traffic-signs-detection-system/total_dataset/total_dataset/images/train'
val_dir = '/kaggle/input/traffic-signs-detection-system/total_dataset/total_dataset/images/val'

print("Training images:")
t=os.listdir(train_dir)
print(len(t))

Training images:
1804


In [8]:
!python train.py --img 640 --batch 16 --epochs 10 --data /kaggle/working/data.yaml --weights yolov5s.pt

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=yolov5s.pt, cfg=, data=/kaggle/working/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=F

In [9]:
!python val.py --weights runs/train/exp/weights/best.pt --data /kaggle/working/data.yaml --img 640

val: data=/kaggle/working/data.yaml, weights=['runs/train/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-371-g6629839d Python-3.10.14 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7031701 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /kaggle/input/traffic-signs-detection-system/total_dataset/total_d
val: WARNING ⚠️ /kaggle/input/traffic-signs-detection-system/total_dataset/total_dataset/images/train/196209192323137.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /kaggle/input/traffic-signs-detection-system/total_dataset/total_dataset/images/train/932229837523918.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /kaggle/input/traffic-signs-detection-system/total_datase

In [10]:
# Move to output directory
!mkdir -p /kaggle/working/weights
!cp runs/train/exp/weights/best.pt /kaggle/working/weights/best.pt

# Or download it directly
import shutil
shutil.make_archive('/kaggle/working/weights', 'zip', '/kaggle/working/weights')


'/kaggle/working/weights.zip'